In [1]:
#Torchtext není k dispozici pro poslední verzi pytorch, budeme tedy využuívat něco jiného ...


In [2]:
from transformers import Trainer, BertForSequenceClassification, BertTokenizer, EarlyStoppingCallback
from datasets import load_from_disk
from torch.utils.data import DataLoader
import torch
import base
import os 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
base.reset_seed()

In [4]:
DATASET = "trec"

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [6]:
train = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

train_aug = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")

In [7]:
tokenizer = BertTokenizer.from_pretrained("gchhablani/bert-base-cased-finetuned-sst2")

In [8]:
train = train.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = train_aug.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

Tokenizing the train dataset:   0%|          | 0/4361 [00:00<?, ? examples/s]

Tokenizing the eval dataset:   0%|          | 0/1091 [00:00<?, ? examples/s]

Tokenizing the test dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing the augmented dataset:   0%|          | 0/67080 [00:00<?, ? examples/s]

In [9]:
base.reset_seed()

In [10]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_fine", logging_dir=f"~/logs/{DATASET}/bert-base_fine", batch_size=128, epochs=10)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [13]:
trainer.train()

/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thre

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.27465716004371643,
 'eval_accuracy': 0.8951744617668894,
 'eval_precision': 0.8931336127022499,
 'eval_recall': 0.8953544607934005,
 'eval_f1': 0.8940722936529324,
 'eval_runtime': 10.0986,
 'eval_samples_per_second': 1333.853,
 'eval_steps_per_second': 10.497,
 'epoch': 10.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert_fine.pth")

In [ ]:
base.reset_seed()

In [ ]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_fine", logging_dir=f"~/logs/{DATASET}/bert-distill_fine", remove_unused_columns=False, batch_size=128, epochs=10, temp=5, lambda_param=.5)

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.118400,2.398580,0.704128,0.707405,0.705165,0.703565
2,2.328100,1.988954,0.759174,0.760040,0.758546,0.758614
3,1.778500,1.836083,0.779817,0.780384,0.779321,0.779441
4,1.469400,1.852803,0.779817,0.784957,0.778522,0.778219
5,1.265300,1.712452,0.786697,0.787427,0.786162,0.786291
6,1.149600,1.704012,0.797018,0.797054,0.796803,0.796877
7,1.073600,1.813213,0.783257,0.786962,0.782152,0.782046
8,1.023000,1.703144,0.801606,0.801831,0.801854,0.801605
9,0.982000,1.723111,0.800459,0.800392,0.800392,0.800392
10,0.962100,1.735673,0.799312,0.799481,0.799013,0.799119


TrainOutput(global_step=4210, training_loss=1.5149904518399273, metrics={'train_runtime': 432.5114, 'train_samples_per_second': 1245.724, 'train_steps_per_second': 9.734, 'total_flos': 401089284540000.0, 'train_loss': 1.5149904518399273, 'epoch': 10.0})

In [ ]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 1.0659942626953125,
 'eval_accuracy': 0.8930215293244247,
 'eval_precision': 0.8909833152005082,
 'eval_recall': 0.8930765995259711,
 'eval_f1': 0.8918784493379457,
 'eval_runtime': 9.935,
 'eval_samples_per_second': 1355.812,
 'eval_steps_per_second': 10.669,
 'epoch': 10.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-distil_fine.pth")

In [ ]:
base.reset_seed()

In [ ]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base-aug_fine", logging_dir=f"~/logs/{DATASET}/bert-base-aug_fine", batch_size=128, epochs=10)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.332100,0.491531,0.795872,0.796140,0.796140,0.795872
2,0.189100,0.606654,0.794725,0.795422,0.794214,0.794355
3,0.150400,0.663262,0.774083,0.776304,0.773186,0.773180
4,0.125700,0.793031,0.770642,0.775226,0.769386,0.769068
5,0.107800,0.880275,0.768349,0.770619,0.767429,0.767389
6,0.096600,0.916054,0.771789,0.772062,0.771396,0.771500
7,0.088300,0.970803,0.764908,0.766803,0.764050,0.764037
8,0.081100,1.057253,0.769495,0.771663,0.768597,0.768575
9,0.077300,1.082565,0.763761,0.766717,0.762714,0.762561
10,0.074800,1.120400,0.763761,0.766457,0.762756,0.762638


TrainOutput(global_step=41690, training_loss=0.13231537677767638, metrics={'train_runtime': 6162.7713, 'train_samples_per_second': 865.893, 'train_steps_per_second': 6.765, 'total_flos': 3972480463800000.0, 'train_loss': 0.13231537677767638, 'epoch': 10.0})

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.25607651472091675,
 'eval_accuracy': 0.9058648849294729,
 'eval_precision': 0.9037956165549527,
 'eval_recall': 0.907060167785301,
 'eval_f1': 0.9050152002729163,
 'eval_runtime': 13.9343,
 'eval_samples_per_second': 966.681,
 'eval_steps_per_second': 7.607,
 'epoch': 10.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-base-aug_fine.pth")

In [ ]:
base.reset_seed()

In [ ]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill-aug_fine", logging_dir=f"~/logs/{DATASET}/bert-distill-aug_fine", remove_unused_columns=False, batch_size=128, epochs=10, temp=5, lambda_param=.5)

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.281900,1.544384,0.811927,0.811859,0.811905,0.811878
2,0.605000,1.664967,0.805046,0.805267,0.804728,0.804845
3,0.467200,1.751891,0.790138,0.792074,0.789330,0.789417
4,0.388100,1.824437,0.790138,0.791278,0.789499,0.789626
5,0.334800,1.868580,0.787844,0.788510,0.787331,0.787461
6,0.301500,1.909845,0.787844,0.788510,0.787331,0.787461
7,0.276100,1.924664,0.782110,0.783860,0.781321,0.781391
8,0.257800,1.961269,0.780963,0.781735,0.780405,0.780524
9,0.246700,1.964471,0.783257,0.784191,0.782658,0.782777
10,0.239800,1.972546,0.784404,0.785122,0.783868,0.783993


TrainOutput(global_step=41690, training_loss=0.43989421490371644, metrics={'train_runtime': 7468.0148, 'train_samples_per_second': 714.554, 'train_steps_per_second': 5.582, 'total_flos': 3972480463800000.0, 'train_loss': 0.43989421490371644, 'epoch': 10.0})

In [ ]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
trainer.evaluate(test)

{'eval_loss': 0.8676739931106567,
 'eval_accuracy': 0.9101707498144024,
 'eval_precision': 0.9083149471174303,
 'eval_recall': 0.9103104024491199,
 'eval_f1': 0.9091869445920846,
 'eval_runtime': 15.0958,
 'eval_samples_per_second': 892.303,
 'eval_steps_per_second': 7.022,
 'epoch': 10.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bert-distil-aug_fine.pth")